In [21]:
import pandas as pd

In [22]:
data = pd.read_csv('.\Project\data.csv')

### Clean the data:
### 1.NaN in category_code
### 2.Inlucde sessions of user_id with at least 1 historical purchase
### 3.Extract only the first component of category_code

In [23]:
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [24]:
data.loc[data['user_id']==410333972]

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
41840087,2019-10-31 12:28:58 UTC,view,1002528,2053013555631882655,electronics.smartphone,apple,617.52,410333972,6c1b04ed-84c5-4330-a012-18853fc63e6f
41841817,2019-10-31 12:30:40 UTC,view,1003304,2053013555631882655,electronics.smartphone,apple,591.25,410333972,6c1b04ed-84c5-4330-a012-18853fc63e6f
41843434,2019-10-31 12:32:13 UTC,cart,1003304,2053013555631882655,electronics.smartphone,apple,591.25,410333972,6c1b04ed-84c5-4330-a012-18853fc63e6f
41848440,2019-10-31 12:37:15 UTC,purchase,1003304,2053013555631882655,electronics.smartphone,apple,591.25,410333972,6c1b04ed-84c5-4330-a012-18853fc63e6f


In [25]:
df = data[data['category_code'].notna()]
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
5,2019-10-01 00:00:05 UTC,view,1480613,2053013561092866779,computers.desktop,pulser,908.62,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9


In [26]:
df.shape

(28933155, 9)

In [27]:
##customers who made at least one purchase:

pdf = df.loc[df['event_type'] == 'purchase',['category_id','user_id']]
pdf.drop_duplicates(inplace=True)
pdf['category_id']=pdf['category_id'].astype(str)

In [28]:
pdf.head()

,category_id,user_id
162,2053013555631882655,543272936
308,2053013555631882655,551377651
442,2053013557418656265,555332717
574,2053013554658804075,524601178
603,2053013554658804075,551377651


In [29]:
trans_data = pdf.groupby('user_id')['category_id'].apply(list).reset_index(name = 'purchased')

In [30]:
trans_data

,user_id,purchased
0,303160429,[2053013553341792533]
1,340041246,[2053013552603594983]
2,384989212,[2053013564003713919]
3,401021311,[2053013554415534427]
4,403013066,[2053013555631882655]
...,...,...
272066,566265448,[2053013553341792533]
272067,566267483,"[2053013553257906447, 2053013552888807671]"
272068,566270060,"[2053013555631882655, 2053013554776244595]"
272069,566274637,[2053013555631882655]


In [31]:
from sklearn.model_selection import train_test_split
from efficient_apriori import apriori
import numpy as np

# No Cluster:

In [32]:
array_trans_data = trans_data.to_numpy()
train, test = train_test_split(array_trans_data,train_size = 0.7,random_state = 6)

In [33]:
train1 = [i for i in train[:,1]]

In [34]:
train1

[['2053013554247762257', '2053013553031414015', '2053013555631882655'],
 ['2053013555631882655'],
 ['2053013563810775923'],
 ['2053013555631882655'],
 ['2053013555631882655'],
 ['2053013555631882655'],
 ['2053013555069845885', '2053013564003713919'],
 ['2053013555631882655'],
 ['2053013555631882655'],
 ['2053013555631882655', '2172371436436455782'],
 ['2053013563810775923'],
 ['2053013555631882655', '2053013554658804075'],
 ['2053013555631882655'],
 ['2053013565782098913'],
 ['2053013563810775923'],
 ['2053013552888807671'],
 ['2053013554658804075'],
 ['2053013555631882655', '2053013561579406073'],
 ['2053013554415534427'],
 ['2053013555095011711',
  '2053013565983425517',
  '2053013559901684381',
  '2053013558316237377'],
 ['2053013554566529381'],
 ['2053013554658804075'],
 ['2053013553970938175'],
 ['2053013555631882655'],
 ['2053013563835941749'],
 ['2053013555631882655'],
 ['2053013555573162395'],
 ['2053013555631882655'],
 ['2053013565782098913', '2053013555631882655'],
 ['2053013

In [35]:
itemsets, rules = apriori(transactions = train1, min_support=0.00018,  min_confidence=0.20)

In [36]:
rules

[{2053013553341792533} -> {2053013555631882655},
 {2053013554155487563} -> {2053013554121933129},
 {2053013554121933129} -> {2053013554155487563},
 {2053013554155487563} -> {2053013554189041997},
 {2053013554415534427} -> {2053013555631882655},
 {2053013554566529381} -> {2053013555631882655},
 {2053013554658804075} -> {2053013555631882655},
 {2053013555573162395} -> {2053013555631882655},
 {2053013555816432043} -> {2053013555631882655},
 {2053013558920217191} -> {2053013555631882655},
 {2053013560530830019} -> {2053013555631882655},
 {2053013560866374351} -> {2053013555631882655},
 {2053013561092866779} -> {2053013555631882655},
 {2053013565069067197} -> {2053013555631882655},
 {2053013565228450757} -> {2053013555631882655},
 {2106075662325383725} -> {2053013555631882655},
 {2116907519078040377} -> {2053013555631882655},
 {2116907525176557699} -> {2053013555631882655},
 {2146660887203676486} -> {2053013555631882655},
 {2172371436436455782} -> {2053013555631882655},
 {205301355894538301

In [37]:
test1 = [i for i in test[:,1]]

In [38]:
total = []
shop = []
for y in rules:
    f = np.array([all(x in i for x in list(y.lhs)) for i in test1])
    g = np.array([all(x in i for x in list(y.lhs)+list(y.rhs)) for i in test1])
    total.append(f)
    shop.append(g)

In [39]:
## prediction accurarcy:
print(' total # of custoemr',len(test1),\
    '\n # of customers can be recommend:',sum(sum(total).astype(bool)),\
      '\n # of customer recommended make the purchase:',sum(sum(shop).astype(bool)),\
     '\n % of success:',round(sum(sum(shop).astype(bool))/sum(sum(total).astype(bool)),4))

 total # of custoemr 81622 
 # of customers can be recommend: 17882 
 # of customer recommended make the purchase: 4064 
 % of success: 0.2273


In [40]:
pdf1 = df.loc[df['event_type'] == 'purchase',['category_id','user_id','price']]
pdf1['category_id']=pdf1['category_id'].astype(str)

###calculate the spending:
#r = []
#for rule in rules:
#    r+=list(rule.rhs)
p = []    
for id,rule in enumerate(rules):
    pdf2 = pdf1.loc[pdf1['category_id'].isin(list(rule.rhs))]
    print('spending: \n',id,sum(pdf2.loc[pdf2['user_id'].isin(test[:,0]*shop[id]),'price']))
    p.append(sum(pdf2.loc[pdf2['user_id'].isin(test[:,0]*shop[id]),'price']))
print('total spend:',sum(p))
#pdf2 = pdf1.loc[pdf1['category_id'].isin(r)]
#print('spending: \n',id, sum(pdf2.loc[pdf2['user_id'].isin(test[:,0]*shop[0]),'price']))

spending: 
 0 2055921.2900000038
spending: 
 1 2789.5299999999993
spending: 
 2 2707.9799999999996
spending: 
 3 1360.1000000000001
spending: 
 4 1348169.2199999944
spending: 
 5 72603.27000000003
spending: 
 6 3361718.550000013
spending: 
 7 364846.3400000006
spending: 
 8 73671.08999999997
spending: 
 9 1398537.6699999955
spending: 
 10 30312.769999999997
spending: 
 11 59932.13000000001
spending: 
 12 221167.38999999987
spending: 
 13 206073.24000000014
spending: 
 14 71049.76999999999
spending: 
 15 23787.450000000004
spending: 
 16 28395.370000000003
spending: 
 17 55520.38999999998
spending: 
 18 18802.6
spending: 
 19 596832.4500000012
spending: 
 20 73600.69
spending: 
 21 18026.820000000003
spending: 
 22 3355.2199999999993
spending: 
 23 92746.47000000006
spending: 
 24 589467.0999999995
spending: 
 25 6192.870000000002
spending: 
 26 179559.53000000026
spending: 
 27 97229.34000000004
spending: 
 28 70492.59
spending: 
 29 69035.27999999997
spending: 
 30 172869.31000000017


In [41]:
pdf[[any(x == i for i in list(rules[0].rhs)) for x in pdf['category_id']]]

,category_id,user_id
162,2053013555631882655,543272936
308,2053013555631882655,551377651
618,2053013555631882655,524325294
860,2053013555631882655,555083442
1558,2053013555631882655,515246296
...,...,...
42446632,2053013555631882655,566278294
42446904,2053013555631882655,555466772
42447378,2053013555631882655,514503441
42447440,2053013555631882655,544501248


# With Clusters:

In [42]:
clusters = pd.read_csv('.\Project\Final Group Assignments2.csv')

In [43]:
clusters.head()

,Group 0,Group 1,Group 2
0,384989212.0,303160429,512363895.0
1,418115333.0,340041246,512372136.0
2,420935067.0,401021311,512398306.0
3,427151532.0,403013066,512411049.0
4,427391662.0,410333972,512445125.0


In [44]:
cluster0 = clusters.loc[clusters['Group 0'].notna(),'Group 0'].astype(int).astype(str)
cluster1 = clusters.loc[clusters['Group 1'].notna(),'Group 1'].astype(int).astype(str)
cluster2 = clusters.loc[clusters['Group 2'].notna(),'Group 2'].astype(int).astype(str)

In [45]:
cluster0

0        384989212
1        418115333
2        420935067
3        427151532
4        427391662
           ...    
84805    566223857
84806    566227497
84807    566238118
84808    566243643
84809    566267483
Name: Group 0, Length: 84810, dtype: object

In [46]:
pdf_c0 = pdf.loc[pdf['user_id'].astype(str).isin(cluster0)]
pdf_c1 = pdf.loc[pdf['user_id'].astype(str).isin(cluster1)]
pdf_c2 = pdf.loc[pdf['user_id'].astype(str).isin(cluster2)]

trans_data_c0 = pdf_c0.groupby('user_id')['category_id'].apply(list).reset_index(name = 'purchased')
trans_data_c1 = pdf_c1.groupby('user_id')['category_id'].apply(list).reset_index(name = 'purchased')
trans_data_c2 = pdf_c2.groupby('user_id')['category_id'].apply(list).reset_index(name = 'purchased')

In [47]:
#c0:
array_trans_data_c0 = trans_data_c0.to_numpy()
train_c0, test_c0 = train_test_split(array_trans_data_c0,train_size = 0.7,random_state = 6)

train_c0_1 = [i for i in train_c0[:,1]]
test_c0_1 = [i for i in test_c0[:,1]]

In [48]:
itemsets0, rules0 = apriori(transactions = train_c0_1, min_support=0.00032,  min_confidence=0.22)

In [49]:
rules0

[{2053013552637149417} -> {2053013552888807671},
 {2053013553341792533} -> {2053013555631882655},
 {2053013553945772349} -> {2053013553970938175},
 {2053013554155487563} -> {2053013554121933129},
 {2053013554121933129} -> {2053013554155487563},
 {2053013554121933129} -> {2053013554189041997},
 {2053013554155487563} -> {2053013554189041997},
 {2053013565069067197} -> {2053013555631882655},
 {2146551843914777018} -> {2053013555631882655},
 {2172371436436455782} -> {2053013555631882655},
 {2053013558945383017} -> {2053013558920217191},
 {2053013563877884791} -> {2053013564003713919},
 {2053013553341792533, 2053013555631882655} -> {2053013554658804075},
 {2053013553341792533, 2053013554658804075} -> {2053013555631882655},
 {2053013553945772349, 2053013555631882655} -> {2053013553970938175},
 {2053013553945772349, 2053013553970938175} -> {2053013555631882655},
 {2053013553970938175, 2053013554658804075} -> {2053013555631882655},
 {2053013554415534427, 2053013554776244595} -> {20530135556318

In [50]:
total0 = []
shop0 = []
for y in rules0:
    f = np.array([all(x in i for x in list(y.lhs)) for i in test_c0_1])
    g = np.array([all(x in i for x in list(y.lhs)+list(y.rhs)) for i in test_c0_1])
    total0.append(f)
    shop0.append(g)

In [51]:
## prediction accurarcy:
print(' total # of custoemr',len(test_c0_1),\
    '\n # of customers can be recommend:',sum(sum(total0).astype(bool)),\
      '\n # of customer recommended make the purchase:',sum(sum(shop0).astype(bool)),\
     '\n % of success:',round(sum(sum(shop0).astype(bool))/sum(sum(total0).astype(bool)),4))

pdf1 = df.loc[df['event_type'] == 'purchase',['category_id','user_id','price']]
pdf1['category_id']=pdf1['category_id'].astype(str)

###calculate the spending:
p = []
for id, rule in enumerate(rules0):
    pdf2 = pdf1.loc[pdf1['category_id'].isin(list(rule.rhs))]
    #print('spending: \n',id,sum(pdf2.loc[pdf2['user_id'].isin(test_c0[:,0]*shop0[id]),'price']))
    p.append(sum(pdf2.loc[pdf2['user_id'].isin(test_c0[:,0]*shop0[id]),'price']))
print('total spend:',sum(p))

 total # of custoemr 25444 
 # of customers can be recommend: 1842 
 # of customer recommended make the purchase: 500 
 % of success: 0.2714
total spend: 1641016.18


In [52]:
#c1:
array_trans_data_c1 = trans_data_c1.to_numpy()
train_c1, test_c1 = train_test_split(array_trans_data_c1,train_size = 0.7,random_state = 6)

train_c1_1 = [i for i in train_c1[:,1]]
test_c1_1 = [i for i in test_c1[:,1]]

In [53]:
itemsets1, rules1 = apriori(transactions = train_c1_1, min_support=0.00031,  min_confidence=0.20)

In [54]:
rules1

[{2053013552293216471} -> {2053013555631882655},
 {2053013552326770905} -> {2053013555631882655},
 {2053013552888807671} -> {2053013555631882655},
 {2053013552955916539} -> {2053013555631882655},
 {2053013553341792533} -> {2053013555631882655},
 {2053013553970938175} -> {2053013553945772349},
 {2053013553945772349} -> {2053013555631882655},
 {2053013553970938175} -> {2053013555631882655},
 {2053013554776244595} -> {2053013554415534427},
 {2053013554415534427} -> {2053013555631882655},
 {2053013563810775923} -> {2053013554415534427},
 {2053013563911439225} -> {2053013554415534427},
 {2053013554566529381} -> {2053013555631882655},
 {2053013554658804075} -> {2053013555631882655},
 {2053013554776244595} -> {2053013555631882655},
 {2053013554834964853} -> {2053013555631882655},
 {2053013555069845885} -> {2053013555631882655},
 {2053013555262783879} -> {2053013555631882655},
 {2053013555321504139} -> {2053013555631882655},
 {2053013555573162395} -> {2053013555631882655},
 {205301355631135994

In [55]:
total1 = []
shop1 = []
for y in rules1:
    f = np.array([all(x in i for x in list(y.lhs)) for i in test_c1_1])
    g = np.array([all(x in i for x in list(y.lhs)+list(y.rhs)) for i in test_c1_1])
    total1.append(f)
    shop1.append(g)

In [56]:
## prediction accurarcy:
print(' total # of custoemr',len(test_c1_1),\
    '\n # of customers can be recommend:',sum(sum(total1).astype(bool)),\
      '\n # of customer recommended make the purchase:',sum(sum(shop1).astype(bool)),\
     '\n % of success:',round(sum(sum(shop1).astype(bool))/sum(sum(total1).astype(bool)),4))

###calculate the spending:
p = []
for id, rule in enumerate(rules1):
    pdf2 = pdf1.loc[pdf1['category_id'].isin(list(rule.rhs))]
    #print('spending: \n',id,sum(pdf2.loc[pdf2['user_id'].isin(test_c0[:,0]*shop0[id]),'price']))
    p.append(sum(pdf2.loc[pdf2['user_id'].isin(test_c1[:,0]*shop1[id]),'price']))
print('total spend:',sum(p))

 total # of custoemr 56099 
 # of customers can be recommend: 14960 
 # of customer recommended make the purchase: 4534 
 % of success: 0.3031
total spend: 14199488.89000002


In [57]:
#c2:
array_trans_data_c2 = trans_data_c2.to_numpy()
train_c2, test_c2 = train_test_split(array_trans_data_c2,train_size = 0.7,random_state = 6)

train_c2_1 = [i for i in train_c2[:,1]]
test_c2_1 = [i for i in test_c2[:,1]]

In [58]:
itemsets2, rules2 = apriori(transactions = train_c2_1, min_support=0.01,  min_confidence=0.5)

In [59]:
rules2

[{2053013553341792533} -> {2053013554658804075},
 {2053013553341792533} -> {2053013555631882655},
 {2053013553341792533} -> {2053013562292437791},
 {2053013554415534427} -> {2053013555631882655},
 {2053013554415534427} -> {2053013562292437791},
 {2053013554658804075} -> {2053013555631882655},
 {2053013554658804075} -> {2053013562292437791},
 {2053013554751078769} -> {2053013562292437791},
 {2053013555069845885} -> {2053013562292437791},
 {2053013555262783879} -> {2053013562292437791},
 {2053013555631882655} -> {2053013562292437791},
 {2053013563810775923} -> {2053013555631882655},
 {2053013558316237377} -> {2053013562292437791},
 {2053013562946749253} -> {2053013562292437791},
 {2053013563810775923} -> {2053013562292437791},
 {2053013563911439225} -> {2053013562292437791},
 {2053013565983425517} -> {2053013562292437791},
 {2053013566176363511} -> {2053013562292437791},
 {2165087460176953468} -> {2053013562292437791},
 {2172371436436455782} -> {2053013562292437791},
 {205301355465880407

In [379]:
total2 = []
shop2 = []
for y in rules2:
    f = np.array([all(x in i for x in list(y.lhs)) for i in test_c2_1])
    g = np.array([all(x in i for x in list(y.lhs)+list(y.rhs)) for i in test_c2_1])
    total2.append(f)
    shop2.append(g)

In [380]:
## prediction accurarcy:
print(' total # of custoemr',len(test_c2_1),\
    '\n # of customers can be recommend:',sum(sum(total2).astype(bool)),\
      '\n # of customer recommended make the purchase:',sum(sum(shop2).astype(bool)),\
     '\n % of success:',round(sum(sum(shop2).astype(bool))/sum(sum(total2).astype(bool)),4))

###calculate the spending:
p = []
for id, rule in enumerate(rules2):
    pdf2 = pdf1.loc[pdf1['category_id'].isin(list(rule.rhs))]
    #print('spending: \n',id,sum(pdf2.loc[pdf2['user_id'].isin(test_c0[:,0]*shop0[id]),'price']))
    p.append(sum(pdf2.loc[pdf2['user_id'].isin(test_c2[:,0]*shop2[id]),'price']))
print('total spend:',sum(p))

 total # of custoemr 80 
 # of customers can be recommend: 19 
 # of customer recommended make the purchase: 19 
 % of success: 1.0
total spend: 1651.1999999999998
